[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20CamemBertForTokenClassification.ipynb)

## Import CamemBertForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 4.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for token classification via `CamembertForTokenClassification` or `TFCamembertForTokenClassification`. These models are usually under `Token Classification` category and have `camembert` in their labels
- Reference: [TFCamembertForTokenClassification](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForTokenClassification)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [Jean-Baptiste/camembert-ner](https://huggingface.co/Jean-Baptiste/camembert-ner) model from HuggingFace as an example
- In addition to `TFCamembertForTokenClassification` we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFCamembertForTokenClassification, CamembertTokenizer 
import tensorflow as tf

MODEL_NAME = 'Jean-Baptiste/camembert-ner'

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFCamembertForTokenClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFCamembertForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


Let's have a look inside these two directories and see what we are dealing with:

In [2]:
!ls -l {MODEL_NAME}

total 889880
-rw-r--r--  1 maziyar  staff        936 Dec 14 21:01 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 14 21:01 saved_model
-rw-r--r--  1 maziyar  staff  440428624 Dec 14 21:01 tf_model.h5


In [3]:
!ls -l {MODEL_NAME}/saved_model/1

total 18808
drwxr-xr-x  2 maziyar  staff       64 Dec 14 21:01 assets
-rw-r--r--  1 maziyar  staff       56 Dec 14 21:01 fingerprint.pb
-rw-r--r--  1 maziyar  staff   166277 Dec 14 21:01 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  9456539 Dec 14 21:01 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 14 21:01 variables


In [4]:
!ls -l {MODEL_NAME}_tokenizer

total 1600
-rw-r--r--  1 maziyar  staff  810912 Dec 14 21:00 sentencepiece.bpe.model
-rw-r--r--  1 maziyar  staff     241 Dec 14 21:00 special_tokens_map.json
-rw-r--r--  1 maziyar  staff     707 Dec 14 21:00 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `sentencepiece.bpe.model` file from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [5]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy sentencepiece.bpe.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {asset_path}

In [6]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [7]:
! ls -l {asset_path}

total 1592
-rw-r--r--  1 maziyar  staff      26 Dec 14 21:01 labels.txt
-rw-r--r--  1 maziyar  staff  810912 Dec 14 21:01 sentencepiece.bpe.model


## Import and Save CamemBertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [8]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.4
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.4


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `CamemBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [10]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = CamemBertForTokenClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
! rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your CamemBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [13]:
! ls -l {MODEL_NAME}_spark_nlp

total 879536
-rw-r--r--  1 maziyar  staff  449510592 Dec 14 21:03 camembert_classification_tensorflow
-rw-r--r--  1 maziyar  staff     810912 Dec 14 21:03 camembert_spp
drwxr-xr-x  4 maziyar  staff        128 Dec 14 21:03 fields
drwxr-xr-x  6 maziyar  staff        192 Dec 14 21:03 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForTokenClassification model 😊 

In [14]:
tokenClassifier_loaded = CamemBertForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'O']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["Je m'appelle jean-baptiste et je vis à montréal"], ['george washington est allé à washington']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

That's it! You can now go wild and use hundreds of `CamemBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀 
